In [1]:
import lsdb
import splusdata
import pandas as pd
from getpass import getpass

In [2]:
# This will print all available links

print(splusdata.get_hipscats())

{'dr4_vacs': {'qso_z': {'hipscats': ['qso_z'], 'margins': ['qso_z_2arcsec']}, 'photozs': {'hipscats': ['photozs'], 'margins': ['photozs_2arcsec']}, 'calib_flag': {'hipscats': ['calib_flag'], 'margins': ['calib_flag_2arcsec']}, 'sqg': {'hipscats': ['sqg'], 'margins': ['sqg_2arcsec']}}, 'sdr1': {'hipscats': ['shorts'], 'margins': ['shorts_2arcsec']}, 'dr4': {'hipscats': ['single', 'dual', 'psf'], 'margins': ['psf_2arcsec', 'single_2arcsec', 'dual_2arcsec']}}


In [3]:
# This will search for a specific pattern
print(splusdata.get_hipscats("dr4/dual"))

[['https://splus.cloud/HIPS/catalogs/dr4/dual/', 'https://splus.cloud/HIPS/catalogs/dr4/dual_2arcsec/']]


In [4]:
from dask.distributed import Client
client = Client(n_workers=10, memory_limit="8GB")
print(client)

<Client: 'tcp://127.0.0.1:38019' processes=10 threads=20, memory=74.51 GiB>


In [6]:
dr4_links = splusdata.get_hipscats("idr4/dual")[0]
dr4_links

['https://splus.cloud/HIPS/catalogs/dr4/dual/',
 'https://splus.cloud/HIPS/catalogs/dr4/dual_2arcsec/']

In [8]:
dr4_margin = lsdb.read_hipscat(dr4_links[1])
for i in dr4_margin.columns:
    print(i)

FileNotFoundError: https://splus.cloud/HIPS/catalogs/dr4/dual_2arcsec/catalog_info.json

In [ ]:
dual = lsdb.read_hipscat(
                dr4_links[0],
                margin_cache=dr4_margin,
                columns = ["Field", "ID", "RA", "DEC", 
                           "X", "Y", "A", "B", "ELLIPTICITY", "ELONGATION",
                           "FWHM", "KRON_RADIUS", "ISOarea", "MU_MAX_r", "MU_MAX_J0660", "MU_MAX_i", "s2n_DET_PStotal",
                           "s2n_g_PStotal", "s2n_J0515_PStotal", "s2n_r_PStotal", "s2n_J0660_PStotal", "s2n_i_PStotal", 
                            "SEX_FLAGS_DET", "SEX_FLAGS_r", "SEX_FLAGS_J0660", "SEX_FLAGS_i",
                            "r_PStotal", "e_r_PStotal",
                            "g_PStotal", "e_g_PStotal",
                            "i_PStotal", "e_i_PStotal",
                            "u_PStotal", "e_u_PStotal",
                            "z_PStotal", "e_z_PStotal",
                            "J0378_PStotal", "e_J0378_PStotal",
                            "J0395_PStotal", "e_J0395_PStotal",
                            "J0410_PStotal", "e_J0410_PStotal",
                            "J0430_PStotal", "e_J0430_PStotal",
                            "J0515_PStotal", "e_J0515_PStotal",
                            "J0660_PStotal", "e_J0660_PStotal",
                            "J0861_PStotal", "e_J0861_PStotal",]
                                        )

### PSF

In [ ]:
dr4_psf = splusdata.get_hipscats("dr4/psf")[0]
psf_margin = lsdb.read_hipscat(dr4_psf[1])
for i in psf_margin.columns:
    print(i)

In [ ]:
psf = lsdb.read_hipscat(
       dr4_psf[0],
       margin_cache=psf_margin,
        columns = ["RA", "DEC", "r_psf", "e_r_psf",
                    "g_psf", "e_g_psf",
                    "i_psf", "e_i_psf",
                    "u_psf", "e_u_psf",
                    "z_psf", "e_z_psf",
                    "J0378_psf", "e_J0378_psf",
                    "J0395_psf", "e_J0395_psf",
                    "J0410_psf", "e_J0410_psf",
                    "J0430_psf", "e_J0430_psf",
                    "J0515_psf", "e_J0515_psf",
                    "J0660_psf", "e_J0660_psf",
                    "J0861_psf", "e_J0861_psf",])

In [ ]:
print(psf.columns) 

#### Matching two hipscat tables

In [ ]:
dual_psf = psf.crossmatch(dual, radius_arcsec = 2)

#### Matching an external table with hipscat

In [ ]:
# GALEX table
df = pd.read_csv("../GUVcat_AISxSDSS_HSmaster.csv") # load your table

In [ ]:
# Scatter the DataFrame to distribute it across workers
df_scattered = client.scatter(df)

In [ ]:
# Load the DataFrame into lsdb
df_hips = lsdb.from_dataframe(df, ra_column="GALEX_RA", dec_column="GALEX_DEC", margin_threshold=3600)

In [ ]:
# matching with dual catalog (instantiated earlier)--dual
matched_table_dual = df_hips.crossmatch(dual, radius_arcsec = 2).compute()

In [ ]:
# Save DataFrame to CSV or other formats
matched_table_dual.to_csv("../GUVcat_AISxSDSS_HSmaster-splusDr4-dual-2arcsec-lsdb.csv", index=False)

In [ ]:
# matching with dual catalog (instantiated earlier)--psf
matched_table_psf = df_hips.crossmatch(psf, radius_arcsec = 2).compute()

In [ ]:
# Save DataFrame to CSV or other formats
matched_table_psf.to_csv("../GUVcat_AISxSDSS_HSmaster-splusDr4-psf-2arcsec-lsdb.csv", index=False)